## Functions for getting article information

We want all articles starting in the beginning of June 2015 until now, and we will gather them in two week periods

In [1]:
all_dates = []
for month in ["01","02","03","04","05","06","07","08","09","10", "11","12"]:
    for date_pair in [("01", "15"), ("16", "31")]:
        date_range = ("2016" + month + date_pair[0], "2016" + month + date_pair[1])
        all_dates.append(date_range)

In [115]:
# for month in ["01", "02", "03", "04", "05", "06", "07", "08"]:
#     for date_pair in [("01", "15"), ("16", "31")]:
#         date_range = ("2016" + month + date_pair[0], "2016" + month + date_pair[1])
#         all_dates.append(date_range)

In [2]:
all_dates

[('20160101', '20160115'),
 ('20160116', '20160131'),
 ('20160201', '20160215'),
 ('20160216', '20160231'),
 ('20160301', '20160315'),
 ('20160316', '20160331'),
 ('20160401', '20160415'),
 ('20160416', '20160431'),
 ('20160501', '20160515'),
 ('20160516', '20160531'),
 ('20160601', '20160615'),
 ('20160616', '20160631'),
 ('20160701', '20160715'),
 ('20160716', '20160731'),
 ('20160801', '20160815'),
 ('20160816', '20160831'),
 ('20160901', '20160915'),
 ('20160916', '20160931'),
 ('20161001', '20161015'),
 ('20161016', '20161031'),
 ('20161101', '20161115'),
 ('20161116', '20161131'),
 ('20161201', '20161215'),
 ('20161216', '20161231')]

In [3]:
import requests
import json
import time
import pandas as pd

In [4]:
def get_articles(search_term, dates):
    url = "https://api.nytimes.com/svc/search/v2/articlesearch.json"
    api_infos = []
    # get all urls
    for start_date, end_date in dates: 
        for page in range(5):
            queries = {
          'api-key': '1484caf76dc448c4bd76cefa335ffe65',
          'q': search_term,
          'begin_date': start_date, 
          'end_date': end_date, 
          'page' : page
            }
            req_t = requests.get(url, params = queries)
#             print(req_t.text)
            time.sleep(6) # delays for 6 seconds
            adict = json.loads(req_t.text)
            doclist = adict['response']['docs']
            for j in range(len(doclist)):
                api_info = ([doclist[j]["web_url"], doclist[j]["headline"],
                      doclist[j]["section_name"],
                      doclist[j]["word_count"], doclist[j]["type_of_material"], 
                      doclist[j]["pub_date"]])
                api_infos.append(api_info)
    # transform data into dataset
    api_dataset = pd.DataFrame(api_infos, columns=['Url','Headline', 
                                           'Section_name', 'Word_Count', 
                                           'Type', 'Pub_Date'])
    return api_dataset

In [5]:
from newspaper import Article

In [6]:
def get_text(url_list):
    articles = []
    article_info=[]
    for address in url_list: 
        try:
            article = Article(address)
            article.download()
            time.sleep(6) # delays for 6 seconds
            article.parse()
            article.nlp()
            article_info = ([article.publish_date, article.authors,
                      article.summary, article.keywords, article.text,
                     article.url])
        except:
            
            pass
        
        articles.append(article_info)
    newspaper_dataset = pd.DataFrame(articles, columns=['Date','Authors', 'Summary', 'Keywords', 'Text', "Url"])
    newspaper_dataset["Text_Decode"] = newspaper_dataset["Text"].apply(lambda x: x.encode('utf-8'))
    print(newspaper_dataset["Text_Decode"])
    newspaper_dataset["Text_Clean"] = newspaper_dataset["Text_Decode"].apply(lambda x: x.decode("utf-8").replace("\n", " "))
    newspaper_dataset = newspaper_dataset.drop('Text_Decode', 1)
    return newspaper_dataset

## Gather India Articles

In [8]:
India_articles = get_articles("modi", all_dates)

In [15]:
Total_India = get_text(India_articles["Url"])

You must `download()` an article before calling `parse()` on it!
You must `download()` an article before calling `parse()` on it!
You must `download()` an article before calling `parse()` on it!
You must `download()` an article before calling `parse()` on it!
You must `download()` an article before calling `parse()` on it!
You must `download()` an article before calling `parse()` on it!
You must `download()` an article before calling `parse()` on it!
You must `download()` an article before calling `parse()` on it!
You must `download()` an article before calling `parse()` on it!
You must `download()` an article before calling `parse()` on it!
You must `download()` an article before calling `parse()` on it!
You must `download()` an article before calling `parse()` on it!
You must `download()` an article before calling `parse()` on it!
You must `download()` an article before calling `parse()` on it!
You must `download()` an article before calling `parse()` on it!
You must `download()` an 

In [10]:
# Pakistan_articles = get_articles("Pakistan", all_dates)

In [15]:
# Pakistan_text = get_text(Pakistan_articles["Url"])

## Clean, Merge, and Pickle Datasets

#### Combine text and api datasets, get rid of duplicates and bad links, and add a column of Text without the "\n"

In [10]:
India_merged = India_articles.merge(Total_India, on = "Url", how = "inner")
good_links = India_merged[India_merged["Text"] != "NYTimes.com no longer supports Internet Explorer 9 or earlier. Please upgrade your browser."]
India_no_duplicates = good_links.drop_duplicates("Text")
India_no_duplicates["Text_Decode"] = India_no_duplicates["Text"].apply(lambda x: x.encode('utf-8'))
India_no_duplicates["Text_Clean"] = India_no_duplicates["Text_Decode"].apply(lambda x: x.decode("utf-8").replace("\n", " "))

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
India_no_duplicates.head(1)

,Url,Headline,Section_name,Word_Count,Type,Pub_Date,Date,Authors,Summary,Keywords,Text,Text_Clean,Text_Decode
0,http://www.nytimes.com/2016/01/06/world/asia/i...,{'print_headline': 'Battle Is Over at Air Base...,World,332,News,2016-01-06T00:00:00Z,2016-01-06,[Hari Kumar],Ammunition recovered from the gunmen is believ...,"[killed, come, gunmen, parrikar, attack, air, ...",Photo\n\nNEW DELHI — All six gunmen who infilt...,Photo NEW DELHI — All six gunmen who infiltra...,b'Photo\n\nNEW DELHI \xe2\x80\x94 All six gunm...


In [18]:
# Pakistan_merged = Pakistan_articles.merge(Pakistan_text, on = "Url", how = "inner")
# Pakistan_no_duplicates = Pakistan_merged.drop_duplicates("Text")
# Pakistan_no_duplicates = Pakistan_no_duplicates[Pakistan_no_duplicates["Text"] != "NYTimes.com no longer supports Internet Explorer 9 or earlier. Please upgrade your browser."]
# final_Pakistan = Pakistan_no_duplicates
# final_Pakistan["Text_Decode"] = final_Pakistan["Text"].apply(lambda x: x.encode('utf-8'))
# final_Pakistan["Text_Clean"] = final_Pakistan["Text_Decode"].apply(lambda x: x.decode("utf-8").replace("\n", " "))

In [19]:
# final_Pakistan.head(1)

In [20]:
# final_Pakistan["India_subject"] = 0
# India_no_duplicates["India_subject"] = 1
# India_Pakistan_dataset = final_Pakistan.append(India_no_duplicates)

In [21]:
# India_Pakistan_dataset.describe()

In [22]:
# India_Pakistan_dataset.head()

In [12]:
import pickle

In [13]:
with open('India_dataset.pkl', 'wb') as picklefile:
    pickle.dump(India_no_duplicates, picklefile,protocol=2)

In [14]:
f = open('India_dataset.pkl', 'rb')
cl = pickle.load(f)
cl

,Url,Headline,Section_name,Word_Count,Type,Pub_Date,Date,Authors,Summary,Keywords,Text,Text_Clean,Text_Decode
0,http://www.nytimes.com/2016/01/06/world/asia/i...,{'print_headline': 'Battle Is Over at Air Base...,World,332,News,2016-01-06T00:00:00Z,2016-01-06,[Hari Kumar],Ammunition recovered from the gunmen is believ...,"[killed, come, gunmen, parrikar, attack, air, ...",Photo\n\nNEW DELHI — All six gunmen who infilt...,Photo NEW DELHI — All six gunmen who infiltra...,b'Photo\n\nNEW DELHI \xe2\x80\x94 All six gunm...
1,http://www.nytimes.com/2016/01/02/opinion/indi...,"{'kicker': 'Editorial', 'print_headline': 'Ind...",Opinion,549,Editorial,2016-01-02T00:00:00Z,2016-01-02,[The Editorial Board],Mr. Sharif has pressed for engagement with Ind...,"[issues, modi, talks, sharif, taliban, pakista...",Photo\n\nAs they clasped hands while walking d...,Photo As they clasped hands while walking dow...,"b""Photo\n\nAs they clasped hands while walking..."
2,http://www.nytimes.com/2016/01/14/world/asia/p...,{'print_headline': 'Pakistan Arrests Militants...,World,689,News,2016-01-14T00:00:00Z,2016-01-14,[Salman Masood],There was no confirmation from Pakistani offic...,"[pakistani, prime, sharif, attack, pakistan, a...","Photo\n\nISLAMABAD, Pakistan — Pakistani offic...","Photo ISLAMABAD, Pakistan — Pakistani officia...","b'Photo\n\nISLAMABAD, Pakistan \xe2\x80\x94 Pa..."
4,http://www.nytimes.com/2016/01/03/business/ene...,"{'print_headline': 'Electrifying India, With t...",Business Day,2421,News,2016-01-03T00:00:00Z,2016-01-03,[Max Bearak],Selco systems typically include a small panel ...,"[model, systems, loans, selco, customers, smal...","The idea behind Selco, and other companies lik...","The idea behind Selco, and other companies lik...","b'The idea behind Selco, and other companies l..."
5,http://www.nytimes.com/2016/01/08/world/asia/m...,"{'main': 'Mufti Mohammad Sayeed, Kashmir Chief...",World,213,Obituary (Obit),2016-01-11T00:00:00Z,2016-01-08,[Agence France-Presse],"Mufti Mohammad Sayeed, chief minister of India...","[party, main, kashmir, peoples, minister, chie...","Mufti Mohammad Sayeed, chief minister of India...","Mufti Mohammad Sayeed, chief minister of India...","b'Mufti Mohammad Sayeed, chief minister of Ind..."
15,http://www.nytimes.com/2016/01/03/world/asia/i...,{'print_headline': '4 Gunmen Die as They Attac...,World,325,News,2016-01-03T00:00:00Z,2016-01-03,[Nida Najar],PhotoNEW DELHI — At least four gunmen were kil...,"[killed, ministry, gunmen, attack, air, india,...",Photo\n\nNEW DELHI — At least four gunmen were...,Photo NEW DELHI — At least four gunmen were k...,"b""Photo\n\nNEW DELHI \xe2\x80\x94 At least fou..."
17,http://www.nytimes.com/2016/01/15/world/asia/i...,{'print_headline': 'World Briefing | Asia; Ind...,World,374,News,2016-01-15T00:00:00Z,2016-01-15,[Hari Kumar],"Vivek Katju, a former Indian ambassador to Afg...","[talks, pakistani, sharif, praising, indias, a...",NEW DELHI — Indian officials on Thursday said ...,NEW DELHI — Indian officials on Thursday said ...,b'NEW DELHI \xe2\x80\x94 Indian officials on T...
29,http://www.nytimes.com/2016/01/04/world/asia/i...,{'print_headline': 'Hunt for Gunmen Continues ...,World,497,News,2016-01-04T00:00:00Z,2016-01-04,[Hari Kumar],"One air force commando was also killed, as wel...","[border, terrorists, gunmen, air, pakistan, fo...",Photo\n\nNEW DELHI — A shootout between Indian...,Photo NEW DELHI — A shootout between Indian s...,b'Photo\n\nNEW DELHI \xe2\x80\x94 A shootout b...
37,http://www.nytimes.com/2016/01/28/opinion/suic...,"{'kicker': 'Editorial', 'print_headline': 'Sui...",Opinion,402,Editorial,2016-01-28T11:30:00Z,2016-01-28,[The Editorial Board],Mr. Vemula was a Dalit — a member of India’s u...,"[suicide, students, modi, members, vemula, stu...","The suicide on Jan. 17 of Rohith Vemula, a Ph....","The suicide on Jan. 17 of Rohith Vemula, a Ph....","b'The suicide on Jan. 17 of Rohith Vemula, a P..."
47,http://www.nytimes.com/2016/01/28/opinion/the-...,"{'k